In [1]:
import findspark
import pandas as pd
findspark.init()

In [2]:
# Starting Session

from pyspark.sql import SparkSession,functions as func
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import year,dayofmonth,monthname,dayname,day,month


spark = SparkSession.builder.master('local').appName('olist_spark').getOrCreate()

In [3]:
# Reading Data , Adding Total Price Column To Data Frame
local_df = spark.read.csv('D:/project/olst/pandas transformation/Transformations/pandas_transformation/Olist_joined_Table_af.csv',inferSchema=True,header=True)
# total_price_per_item_df = local_df.withColumn('total_price_per_item',func.col('price')+func.col('freight_value')) # delete

In [4]:
local_df.schema

StructType([StructField('index', IntegerType(), True), StructField('order_id', StringType(), True), StructField('customer_id', StringType(), True), StructField('order_status', StringType(), True), StructField('order_purchase_timestamp', TimestampType(), True), StructField('order_approved_at', TimestampType(), True), StructField('order_delivered_carrier_date', TimestampType(), True), StructField('order_delivered_customer_date', TimestampType(), True), StructField('order_estimated_delivery_date', TimestampType(), True), StructField('order_item_id', DoubleType(), True), StructField('product_id', StringType(), True), StructField('seller_id', StringType(), True), StructField('seller_limit_date_to_ship_to_vendor', TimestampType(), True), StructField('price', DoubleType(), True), StructField('freight_value', DoubleType(), True), StructField('customer_unique_id', StringType(), True), StructField('customer_city', StringType(), True), StructField('customer_state', StringType(), True), StructFiel

In [ ]:
# StructType([StructField('index', IntegerType(), True), StructField('order_id', StringType(), True), 
#             StructField('customer_id', StringType(), True), StructField('order_status', StringType(), True), 
#             StructField('order_purchase_timestamp', TimestampType(), True), 
#             StructField('order_approved_at', TimestampType(), True), 
#             StructField('order_delivered_carrier_date', TimestampType(), True), 
#             StructField('order_delivered_customer_date', TimestampType(), True), 
#             StructField('order_estimated_delivery_date', TimestampType(), True), 
#             StructField('order_item_id', DoubleType(), True), 
#             StructField('product_id', StringType(), True), 
#             StructField('seller_id', StringType(), True), 
#             StructField('seller_limit_date_to_ship_to_vendor', TimestampType(), True), 
#             StructField('price', DoubleType(), True), 
#             StructField('freight_value', DoubleType(), True), 
#             StructField('customer_unique_id', StringType(), True), 
#             StructField('customer_city', StringType(), True), 
#             StructField('customer_state', StringType(), True), 
#             StructField('payment_sequential', DoubleType(), True), 
#             StructField('payment_type', StringType(), True),
#             StructField('payment_installments', DoubleType(), True), 
#             StructField('payment_value', DoubleType(), True), 
#             StructField('product_category_name', StringType(), True), 
#             StructField('seller_city', StringType(), True), 
#             StructField('seller_state', StringType(), True)])

In [4]:
# Adding Column delivery_time_by_day [order_purchase_timestamp - order_delivered_customer_date] in day scoope
# and filling nulls with zero

delivery_time_df = local_df.withColumn('is_success',
            func.when(func.lower(func.col('order_status')).isin("delivered","shipped","invoiced"), 1).otherwise(0)).\
            na.fill('not_defined','payment_type')
delivery_time_df.show(5)

+-----+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+-------------+--------------------+--------------------+-----------------------------------+-----+-------------+--------------------+--------------------+--------------+------------------+------------+--------------------+-------------+---------------------+-----------+------------+----------+
|index|            order_id|         customer_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|order_item_id|          product_id|           seller_id|seller_limit_date_to_ship_to_vendor|price|freight_value|  customer_unique_id|       customer_city|customer_state|payment_sequential|payment_type|payment_installments|payment_value|product_category_name|seller_city|seller_state|is_success|
+-----+-----------

In [5]:
# Fix Payment_type if is_success == 0 the payment = 0 + if is_success == 1 and payment = null then paymen = (item + freight)
payment_type_fix_df = delivery_time_df.withColumn('payment_values',\
                                                   func.when(func.col('is_success') == 0, 0).\
                                                    when(func.col('payment_value').isNull() & \
                                                    (func.col('is_success') == 1), func.col('price') + func.col('freight_value')                                                                          
                                                        ).otherwise(func.col('payment_value'))
                                                ).drop(func.col('payment_value'))
payment_type_fix_df.show(5)

+-----+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+-------------+--------------------+--------------------+-----------------------------------+-----+-------------+--------------------+--------------------+--------------+------------------+------------+--------------------+---------------------+-----------+------------+----------+--------------+
|index|            order_id|         customer_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|order_item_id|          product_id|           seller_id|seller_limit_date_to_ship_to_vendor|price|freight_value|  customer_unique_id|       customer_city|customer_state|payment_sequential|payment_type|payment_installments|product_category_name|seller_city|seller_state|is_success|payment_values|
+-----+---------

In [6]:
# fix nulls in order_delivered_customer_date if is_success = 1 & date = null then date = carrierdate + 2 days
order_delivered_date_fix = payment_type_fix_df.withColumn\
        ('orders_delivered_customer_date',func.when((func.col("is_success") == 1) &\
        (func.col('order_delivered_customer_date').isNull()) &\
        (func.col('order_delivered_carrier_date').isNull()),\
        func.date_add(start=func.col('order_estimated_delivery_date'),days=2)).\
        when((func.col("is_success") == 1) & (func.col('order_delivered_customer_date').isNull()),
        func.date_add(start=func.col('order_delivered_carrier_date'),days=2)).\
        otherwise(func.col('order_delivered_customer_date'))).drop('order_delivered_customer_date')\
                        .na.fill('Missing',subset=['seller_city','seller_state','customer_city','customer_state','product_category_name'])
order_delivered_date_fix.show(5,False)

+-----+--------------------------------+--------------------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-------------+--------------------------------+--------------------------------+-----------------------------------+-----+-------------+--------------------------------+--------------------+--------------+------------------+------------+--------------------+---------------------+-----------+------------+----------+--------------+------------------------------+
|index|order_id                        |customer_id                     |order_status|order_purchase_timestamp|order_approved_at  |order_delivered_carrier_date|order_estimated_delivery_date|order_item_id|product_id                      |seller_id                       |seller_limit_date_to_ship_to_vendor|price|freight_value|customer_unique_id              |customer_city       |customer_state|payment_sequential|payment_type|payment_installments

In [7]:
# adding column delivery_delay to see how many days did spent until the customer recieved the package
# adding column is_delayed to see was the package delayed or no 

delivery_delay_df = order_delivered_date_fix.\
withColumn('days_passed_for_delivery',\
    func.timestamp_diff('Day',func.col('order_purchase_timestamp')\
        ,func.col('orders_delivered_customer_date'))).\
withColumn('delivery_delaied',func.date_diff(func.col('orders_delivered_customer_date'),func.col('order_estimated_delivery_date')))\
.withColumn('is_delayed',func.when(func.col('delivery_delaied') > 0, 'True').otherwise('False'))

In [74]:
delivery_delay_df.show(10)

+-----+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-------------+--------------------+--------------------+-----------------------------------+-----+-------------+--------------------+--------------------+--------------+------------------+------------+--------------------+---------------------+-----------+------------+----------+--------------+------------------------------+------------------------+----------------+----------+
|index|            order_id|         customer_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_estimated_delivery_date|order_item_id|          product_id|           seller_id|seller_limit_date_to_ship_to_vendor|price|freight_value|  customer_unique_id|       customer_city|customer_state|payment_sequential|payment_type|payment_installments|product_category_name|seller_city|seller_state|is_success|payment_

In [8]:
# adding dates and time to order_purchase for more insights
dates_df = delivery_delay_df.\
withColumn('order_purchase_hour',func.hour('order_purchase_timestamp')).\
withColumn('order_purchase_dayname', func.dayname('order_purchase_timestamp')).\
withColumn('order_purchase_day',func.day('order_purchase_timestamp')).\
withColumn('order_purchase_month',func.month('order_purchase_timestamp')).\
withColumn('order_purchase_monthname',func.monthname('order_purchase_timestamp')).\
withColumn('order_purchase_year',func.year('order_purchase_timestamp'))

dates_df.show(5)

+-----+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-------------+--------------------+--------------------+-----------------------------------+-----+-------------+--------------------+--------------------+--------------+------------------+------------+--------------------+---------------------+-----------+------------+----------+--------------+------------------------------+------------------------+----------------+----------+-------------------+----------------------+------------------+--------------------+------------------------+-------------------+
|index|            order_id|         customer_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_estimated_delivery_date|order_item_id|          product_id|           seller_id|seller_limit_date_to_ship_to_vendor|price|freight_value|  customer_unique_id|       customer_city|custom

In [9]:
# count of order status
order_status_df = local_df.groupBy(local_df.order_status).agg(func.count('*').alias('Total_Status')).\
sort('Total_Status',ascending=False)
order_status_df.show()

+------------+------------+
|order_status|Total_Status|
+------------+------------+
|   delivered|      116689|
|     shipped|        1275|
|    canceled|         762|
| unavailable|         650|
|  processing|         388|
|    invoiced|         385|
|     created|           5|
|    approved|           3|
+------------+------------+



In [19]:
# most sold category
revenue_per_category_df = payment_type_fix_df.where(func.col('is_success')==1).\
    groupBy(payment_type_fix_df.product_category_name).agg(func.round(func.sum('payment_values'),3)\
        .alias('total_revenue_per_category')).orderBy('total_revenue_per_category',ascending=False)
revenue_per_category_df.show(10,False)

+---------------------+--------------------------+
|product_category_name|total_revenue_per_category|
+---------------------+--------------------------+
|bed_bath_table       |1710493.33                |
|health_beauty        |1651700.03                |
|computers_accessories|1567243.48                |
|watches_gifts        |1415384.5                 |
|furniture_decor      |1408566.91                |
|sports_leisure       |1376032.55                |
|housewares           |1082688.79                |
|auto                 |840964.31                 |
|garden_tools         |820347.64                 |
|cool_stuff           |755756.61                 |
+---------------------+--------------------------+
only showing top 10 rows


In [20]:
# most sold product
revenue_per_product_df = payment_type_fix_df.where(func.col('is_success')==1).\
    groupBy(payment_type_fix_df.product_id).agg(func.round(func.sum('payment_values'),3)\
        .alias('total_revenue_per_product')).orderBy('total_revenue_per_product',ascending=False)
revenue_per_product_df.show(10,False)

+--------------------------------+-------------------------+
|product_id                      |total_revenue_per_product|
+--------------------------------+-------------------------+
|5769ef0a239114ac3a854af00df129e4|109312.64                |
|bb50f2e236e5eea0100680137654686c|81887.42                 |
|422879e10f46682990de24d770e7f83d|79512.22                 |
|d1c427060a0f73f6b889a5c7c61f2ac4|70179.16                 |
|6cdd53843498f92890544667809f1595|64825.67                 |
|d5991653e037ccb7af6ed7d94246b249|64143.26                 |
|aca2eb7d00ea1a7b8ebd4e68314663af|63788.12                 |
|99a4788cb24856965c36a24e339b6058|63053.4                  |
|a62e25e09e05e6faf31d90c6ec1aa3d1|63042.85                 |
|3dd2a17168ec895c781a9191c1e95ad7|58962.14                 |
+--------------------------------+-------------------------+
only showing top 10 rows


In [21]:
# most selling sellers

seller_revenue_df = delivery_delay_df.where(func.col('is_success')==1).\
    groupBy('seller_id').\
        agg(func.round(func.sum('payment_values'),3).alias('total_revenue_per_seller')).\
        orderBy('total_revenue_per_seller',ascending=False)
seller_revenue_df.show(5,False)

+--------------------------------+------------------------+
|seller_id                       |total_revenue_per_seller|
+--------------------------------+------------------------+
|7c67e1448b00f6e969d365cea6b010ab|507166.91               |
|1025f0e2d44d7041d6cf58b6550e0bfa|308222.04               |
|4a3ca9315b744ce9f8e9374361493884|301064.61               |
|1f50f920176fa81dab994f9023523100|290185.92               |
|53243585a1d6dc2643021fd1853d8905|284628.19               |
+--------------------------------+------------------------+
only showing top 5 rows


In [22]:
# most buying customers
royal_customers_df  = delivery_delay_df.where(func.col('is_success')==1).\
    groupBy('customer_id').\
        agg(func.round(func.sum('payment_values'),3).alias('total_revenue_per_customer')).\
        orderBy('total_revenue_per_customer',ascending=False)
royal_customers_df.show(5,False)

+--------------------------------+--------------------------+
|customer_id                     |total_revenue_per_customer|
+--------------------------------+--------------------------+
|1617b1357756262bfa56ab541c47bc16|109312.64                 |
|bd5d39761aa56689a265d95d8d32b8be|45256.0                   |
|be1b70680b9f9694d8c70f41fa3dc92b|44048.0                   |
|05455dfa7cd02f13d132aa7a6a9729c6|36489.24                  |
|1ff773612ab8934db89fd5afa8afe506|30186.0                   |
+--------------------------------+--------------------------+
only showing top 5 rows


In [25]:
# most used payment method
payment_method_df = delivery_time_df.where(func.col('is_success')==1).\
    groupBy('payment_type').agg(func.countDistinct('order_id').alias('total_payment_method')).\
        orderBy('total_payment_method',ascending=False)
payment_method_df.show(payment_method_df.count())
# null ---> not defined

+------------+--------------------+
|payment_type|total_payment_method|
+------------+--------------------+
| credit_card|               75391|
|      boleto|               19467|
|     voucher|                3734|
|  debit_card|                1513|
| not_defined|                   1|
+------------+--------------------+



In [26]:
# most demanding city
city_df = order_delivered_date_fix.where(func.col('is_success')==1).\
    groupBy('customer_city').agg(func.countDistinct('order_id').alias('most_demanding_city')).\
        orderBy('most_demanding_city',ascending=False)
city_df.show(city_df.count())

+--------------------+-------------------+
|       customer_city|most_demanding_city|
+--------------------+-------------------+
|           sao paulo|              15238|
|      rio de janeiro|               6785|
|      belo horizonte|               2730|
|            brasilia|               2104|
|            curitiba|               1500|
|            campinas|               1419|
|        porto alegre|               1368|
|            salvador|               1230|
|           guarulhos|               1165|
|sao bernardo do c...|                918|
|             niteroi|                839|
|         santo andre|                785|
|              osasco|                734|
|              santos|                704|
| sao jose dos campos|                680|
|             goiania|                678|
|           fortaleza|                644|
|            sorocaba|                619|
|              recife|                607|
|       florianopolis|                561|
|          

In [28]:
# Adding Insight: Total Revenue Per Order
total_price_per_order_df = payment_type_fix_df.where(func.col('is_success')==1).groupBy('order_id').\
    agg(func.round(func.sum('payment_values'),3).alias('order_revenue')).\
        orderBy('order_revenue',ascending=False)
total_price_per_order_df.show(10)

+--------------------+-------------+
|            order_id|order_revenue|
+--------------------+-------------+
|03caa2c082116e1d3...|    109312.64|
|ab14fdcfbe524636d...|      45256.0|
|1b15974a0141d54e3...|      44048.0|
|2cc9089445046817a...|     36489.24|
|e8fa22c3673b1dd17...|      30186.0|
|736e1922ae60d0d6a...|     29099.52|
|9f5054bd9a3c71702...|      26133.8|
|9aec4e1ae90b23c7b...|      22346.6|
|71dab1155600756af...|     21874.05|
|912343626f370ead5...|     19457.04|
+--------------------+-------------+
only showing top 10 rows


In [ ]:
# Saving df to csvs
# 1- full data frame 
dates_df.coalesce(1).write.csv('D:/project/olst/pandas transformation/Transformations/Spark_transfromation/Spark_saves/olist_full.csv',mode='overwrite',header=True)

# insights data frame
order_status_df.coalesce(1).write.csv('D:/project/olst/pandas transformation/Transformations/Spark_transfromation/Spark_saves/order_status.csv',mode='overwrite',header=True)
revenue_per_category_df.coalesce(1).write.csv('D:/project/olst/pandas transformation/Transformations/Spark_transfromation/Spark_saves/revenue_per_category.csv',mode='overwrite',header=True)
revenue_per_product_df.coalesce(1).write.csv('D:/project/olst/pandas transformation/Transformations/Spark_transfromation/Spark_saves/revenue_per_product.csv',mode='overwrite',header=True)
seller_revenue_df.coalesce(1).write.csv('D:/project/olst/pandas transformation/Transformations/Spark_transfromation/Spark_saves/seller_revenue.csv',mode='overwrite',header=True)
royal_customers_df.coalesce(1).write.csv('D:/project/olst/pandas transformation/Transformations/Spark_transfromation/Spark_saves/royal_customers.csv',mode='overwrite',header=True)
payment_method_df.coalesce(1).write.csv('D:/project/olst/pandas transformation/Transformations/Spark_transfromation/Spark_saves/payment_method.csv',mode='overwrite',header=True)
city_df.coalesce(1).write.csv('D:/project/olst/pandas transformation/Transformations/Spark_transfromation/Spark_saves/city.csv',mode='overwrite',header=True)
total_price_per_order_df.coalesce(1).write.csv('D:/project/olst/pandas transformation/Transformations/Spark_transfromation/Spark_saves/total_price_per_order.csv',mode='overwrite',header=True)

In [29]:
frames = [dates_df, order_status_df, revenue_per_category_df, revenue_per_product_df, seller_revenue_df, royal_customers_df, payment_method_df, city_df, total_price_per_order_df]

frame_names = [
    'dates_analysis', 'order_status_summary', 'revenue_by_category', 
    'revenue_by_product', 'seller_revenue_summary', 'royal_customers', 
    'payment_method_summary', 'city_analysis', 'total_price_per_order'
]

path = "D:/project/olst/pandas transformation/Transformations/Spark_transfromation/"

for frame, name in zip(frames, frame_names):
    try:
        pandas_df = frame.coalesce(1).toPandas()
        
        file_path = f"{path}{name}.csv"
        
        pandas_df.to_csv(file_path, index=False)
        
        print(f"Saving Finished For ----> {name}.csv")
        
    except Exception as e:
        print(f" Error in DataFrame -----x {name}")
        print(f" Error : {e}")

Saving Finished For ----> dates_analysis.csv
Saving Finished For ----> order_status_summary.csv
Saving Finished For ----> revenue_by_category.csv
Saving Finished For ----> revenue_by_product.csv
Saving Finished For ----> seller_revenue_summary.csv
Saving Finished For ----> royal_customers.csv
Saving Finished For ----> payment_method_summary.csv
Saving Finished For ----> city_analysis.csv
Saving Finished For ----> total_price_per_order.csv


In [30]:
spark.stop()